In [2]:
import torch
import torchvision
from torchvision import models,datasets,transforms
import os
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np

In [3]:
dir = '../data/dataset_split'

params = { 'batch_size':16,
           'shuffle':True,
           'num_workers':4 }


transform = transforms.Compose([transforms.Resize(256),
                                transforms.RandomResizedCrop(256), #Augmented
                                transforms.ToTensor(),
                                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])
train_dataset = datasets.ImageFolder(os.path.join(dir, 'train'),transform = transform )

transform = transforms.Compose([transforms.Resize(256),
                                transforms.CenterCrop(256),
                                transforms.ToTensor(),
                                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])
val_dataset = datasets.ImageFolder(os.path.join(dir, 'val'),transform = transform )

train_dataloader = torch.utils.data.DataLoader(train_dataset, **params)
val_dataloader = torch.utils.data.DataLoader(val_dataset, **params)

class_names = train_dataset.classes

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [4]:
print('Train dataset = {}\n'.format(len(train_dataset)),'Val dataset = {}'.format(len(val_dataset)))
print('Classes = {}'.format(class_names))

Train dataset = 2019
 Val dataset = 508
Classes = ['cardboard', 'glass', 'metal', 'paper', 'plastic', 'trash']


In [6]:
import torch
import pandas as pd
import csv

def train(model, loss_fn, optimizer, num_epochs=25, save_path='training_trashnet_results.csv'):
    
    best_acc = 0
    results = []  # Store epoch-wise results as lists of arrays

    for epoch in range(num_epochs):
        
        print('Epoch {}'.format(epoch+1))
        
        # Train dataset
        model.train()
        train_loss = 0.0
        train_correct = 0
        size = len(train_dataset)
        for inputs, labels in train_dataloader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()  # Zero the gradients
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            loss = loss_fn(outputs, labels)
            loss.backward()
            optimizer.step()
            train_loss += loss.item() * inputs.size(0)
            train_correct += torch.sum(preds == labels.data)
            
        train_loss = train_loss / size
        train_acc = train_correct.double() / size
            
        print('Training Loss: {:.4f} Acc: {:.4f}'.format(train_loss, train_acc))
        
        # Validation dataset
        model.eval()
        val_loss = 0.0
        val_correct = 0
        size = len(val_dataset)
        for inputs, labels in val_dataloader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            with torch.no_grad():
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = loss_fn(outputs, labels)
            val_loss += loss.item() * inputs.size(0)
            val_correct += torch.sum(preds == labels.data) 

        val_loss = val_loss / size
        val_acc = val_correct.double() / size    

        print('Validation Loss: {:.4f} Acc: {:.4f}'.format(val_loss, val_acc))    
            
        if val_acc > best_acc:
            best_acc = val_acc

        # Store the results for this epoch as a list of arrays
        results.append([epoch + 1, [train_loss], [train_acc.item()], [val_loss], [val_acc.item()]])
    
    # Save results to CSV
    df = pd.DataFrame(results, columns=['Epoch', 'Train Loss', 'Train Accuracy', 'Validation Loss', 'Validation Accuracy'])
    df.to_csv(save_path, index=False)
    
    print('Best Validation Accuracy: {:.4f}'.format(best_acc))
    print(f'Training results saved to {save_path}')
    
    return model

In [7]:
net = torchvision.models.densenet121(pretrained=True)
for param in net.parameters():
    param.requires_grad = False # freeze all the weights

ft = net.classifier.in_features # final layer of the densenet
net.classifier = nn.Linear(ft, 6) # new layer according to our dataset with weights unfrozen

net = net.to(device)

loss = nn.CrossEntropyLoss()

# only final layer optimized
optimizer = optim.SGD(net.classifier.parameters(), lr=0.0001, momentum=0.9) 

c:\Users\vidia\anaconda3\envs\mlp\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\vidia\anaconda3\envs\mlp\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet121_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
net = train(net,loss,optimizer,num_epochs=100)



Epoch 1
Training Loss: 1.6500 Acc: 0.3214
Validation Loss: 1.5167 Acc: 0.4843
Epoch 2
Training Loss: 1.4459 Acc: 0.5077
Validation Loss: 1.3265 Acc: 0.6102
Epoch 3
Training Loss: 1.3073 Acc: 0.5810
Validation Loss: 1.2077 Acc: 0.6280
Epoch 4
Training Loss: 1.2018 Acc: 0.6290
Validation Loss: 1.1008 Acc: 0.6811
Epoch 5
Training Loss: 1.1290 Acc: 0.6558
Validation Loss: 1.0329 Acc: 0.6870
Epoch 6
Training Loss: 1.0661 Acc: 0.6751
Validation Loss: 0.9757 Acc: 0.6988
Epoch 7
Training Loss: 1.0339 Acc: 0.6825
Validation Loss: 0.9172 Acc: 0.7224
Epoch 8
Training Loss: 0.9960 Acc: 0.6810
Validation Loss: 0.8794 Acc: 0.7224
Epoch 9
Training Loss: 0.9488 Acc: 0.7088
Validation Loss: 0.8506 Acc: 0.7264
Epoch 10
Training Loss: 0.9363 Acc: 0.6954
Validation Loss: 0.8329 Acc: 0.7303
Epoch 11
Training Loss: 0.8922 Acc: 0.7226
Validation Loss: 0.8089 Acc: 0.7402
Epoch 12
Training Loss: 0.9055 Acc: 0.7048
Validation Loss: 0.7920 Acc: 0.7480
Epoch 13
Training Loss: 0.8701 Acc: 0.7236
Validation Loss: 0

Saving the parameters of the desnet model after training!

In [ ]:
import torch

torch.save(net.state_dict(), "trained_trashnet_densenet.pth")  # Save model weights
print("Model saved as trained_densenet.pth")


Model saved as trained_densenet.pth
